In [ ]:
# pgeocode

In [1]:
import pandas as pd
import numpy as np
import re
import pgeocode

from fuzzywuzzy import process
from fuzzywuzzy import fuzz


In [2]:
master_co_data = pd.read_excel('master_co_data.xlsx', header=0)

working_master = master_co_data.copy()

In [3]:
# Processing to create valid 'pin' file and no 'pin' file

yes_pin_master = working_master[working_master['PIN'].notna()].reset_index(drop=True) # master with some value in PIN
nan_pin_master = working_master[working_master['PIN'].isna()].reset_index(drop=True)  # master with NaN value in PIN

yes_pin_master['PIN'] = yes_pin_master['PIN'].astype(str)                      # converting pin to string

yes_pin_master['PIN'] = yes_pin_master['PIN'].apply(lambda x: re.sub('[^0-9]', '', x))  # 'pins' with only digits
yes_pin_6ch = yes_pin_master.loc[yes_pin_master['PIN'].str.len() == 6]         # pin should be of 6 digits
yes_pin_not_6ch = yes_pin_master.loc[yes_pin_master['PIN'].str.len() != 6]

invalid_pin = yes_pin_6ch[yes_pin_6ch['PIN'].str.startswith(('0'))]            # pins cannot start with '0'
valid_pin = yes_pin_6ch[yes_pin_6ch['PIN'].str.startswith(('1', '2', '3', '4', '5', '6', '7', '8', '9'))]

frames_1 = [invalid_pin, yes_pin_not_6ch]                                      # merging invalid_pin dataframes
merged_invalid_pin = pd.concat(frames_1).reset_index(drop=True)
merged_invalid_pin['PIN'] = ''                                                 # convering 'pin' column to blank and then to NaN
merged_invalid_pin['PIN'] = merged_invalid_pin['PIN'].replace('', np.nan, regex=True)

frames_2 = [nan_pin_master, merged_invalid_pin]                      # merging invalid_pin having pin = Nan with earlier nan_pin
merged_nan_pin = pd.concat(frames_2).reset_index(drop=True)

print(len(working_master))
print(len(valid_pin))
print(len(merged_nan_pin))

686829
464219
222610


In [4]:
merged_nan_pin['ADD'] = merged_nan_pin['ADD'].astype(str).str.lower()       # making all in lower case
merged_nan_pin['PIN'] = merged_nan_pin['PIN'].astype(str)           # making type of column 'PIN' from 'float' to 'str'

merged_nan_pin['PIN'] = merged_nan_pin['ADD'].str.extract(r"\b([1-9]{1}[0-9]{2}\s{0,1}[0-9]{3})\b")  # take out pincode from
                                                      # the address. Here blank pins automatically gets converted to Nan(float)
merged_nan_pin['PIN'] = merged_nan_pin['PIN'].str.replace(' ', '')         # removing gap between digits of pincode
                                                                           # i.e. from 396 191 to 396191

next_nan_pin = merged_nan_pin[merged_nan_pin['PIN'].isna()].reset_index(drop=True)
next_yes_pin = merged_nan_pin[~merged_nan_pin['PIN'].isna()].reset_index(drop=True)

# Here checking for 6 digits and checking for starting with '0' not reqyured since above regex takes care of this

frames_3 = [valid_pin, next_yes_pin]
final_valid_pin = pd.concat(frames_3).reset_index(drop=True)

final_invalid_pin = next_nan_pin

print(len(working_master))
print(len(final_valid_pin))
print(len(final_invalid_pin))

686829
626554
60275


In [140]:
# Going for geocode :

valid_pin_df = final_valid_pin.copy()
valid_pin_df = valid_pin_df.fillna(0)    # filling all NaN with  zero for subsequent 'if' statement

valid_pin_with_city_state = valid_pin_df[(valid_pin_df[['CITY','STATE']] != 0).all(axis=1)].reset_index(drop=True)
valid_pin_wout_city_state = valid_pin_df[~(valid_pin_df[['CITY','STATE']] != 0).all(axis=1)].reset_index(drop=True)

#========================================================================================================================

geocode_df = valid_pin_wout_city_state.copy()
#geocode_df = zz[3000:3005].reset_index(drop=True)

for i in range(len(geocode_df)):

    nomi = pgeocode.Nominatim('in')
    pin_code = geocode_df['PIN'][i]
    output_data = nomi.query_postal_code(pin_code)

    output_data.replace(np.nan, 0, inplace=True)
    if output_data[1] != 0:
#        city = output_data[2] + ',' + output_data[7] + ',' + output_data[5]
        city = output_data[5]
        state = output_data[3]
        if geocode_df['CITY'][i] == 0:
            geocode_df['CITY'][i] = city
        if geocode_df['STATE'][i] == 0:
            geocode_df['STATE'][i] = state
    
#    print('=======================',i)
#==========================================================================================================================

# If pin is wrong i.e. does not exist, this will not give any result and hence, 'city' or 'state' will remain '0' as 
# original. To take care of this, following two lines

geocode_with_city_state = geocode_df[(geocode_df[['CITY', 'STATE']] != 0).all(axis=1)]
geocode_wout_city_state = nan_pin_with_pin[~(nan_pin_with_pin[['CITY', 'STATE']] != 0).all(axis=1)]

geocode_wout_city_state = ''                                              # convering 'pin' column to blank and then to NaN
geocode_wout_city_state['PIN'] = geocode_wout_city_state['PIN'].replace('', np.nan, regex=True)


frames_4 = [valid_pin_with_city_state, geocode_with_city_state]
df_with_city_pin_state = pd.concat(frames_4).reset_index(drop=True)  ## OUTPUT-1 : master file

frames_5 = [final_invalid_pin, geocode_wout_city_state]
df_with_invalid_pin = pd.concat(frames_5).reset_index(drop=True)     ## OUTPUT-2 : to compare with city_master and edit


<ipython-input-140-42663edb2f54>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geocode_df['CITY'][i] = city
<ipython-input-140-42663edb2f54>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geocode_df['STATE'][i] = state


======================= 0
======================= 1
======================= 2
======================= 3
======================= 4
4


In [144]:
#motitalav,near ram madhi, kumbhawada, bhavnagar, bhavnagar - 364006, gujarat, india

idd = valid_pin_wout_city_state.index[valid_pin_wout_city_state['ADD'] == 'motitalav,near ram madhi, kumbhawada, bhavnagar, bhavnagar - 364006, gujarat, india']
idd

Int64Index([123718], dtype='int64')

In [5]:
invalid_pin_0 = final_invalid_pin.fillna(0)

invalid_pin_with_citystate = invalid_pin_0[(invalid_pin_0[['CITY','STATE']] != 0).all(axis=1)].reset_index(drop=True)
invalid_pin_wout_city_state = invalid_pin_0[~(invalid_pin_0[['CITY','STATE']] != 0).all(axis=1)].reset_index(drop=True)

invalid_pin_wout_citystate = invalid_pin_wout_city_state[(invalid_pin_wout_city_state[['CITY','STATE']] == 0).all(axis=1)].reset_index(drop=True)
invalid_pin_wout_cityORstate = invalid_pin_wout_city_state[~(invalid_pin_wout_city_state[['CITY','STATE']] == 0).all(axis=1)].reset_index(drop=True)

invalid_pin_wout_state = invalid_pin_wout_cityORstate[(invalid_pin_wout_cityORstate[['STATE']] == 0).all(axis=1)].reset_index(drop=True)
invalid_pin_wout_city = invalid_pin_wout_cityORstate[(invalid_pin_wout_cityORstate[['CITY']] == 0).all(axis=1)].reset_index(drop=True)

frames = [invalid_pin_wout_citystate, invalid_pin_wout_city]   # here city is to be found out from address
invalid_pin_check_city = pd.concat(frames).reset_index(drop=True)

# invalid_pin_with_citystate : nothing needs to be done, invalid_pin_wout_state : state to find from city,
# invalid_pin_check_city : city to find from address and state to find from city.

#invalid_pin_wout_city_state.tail(3)
print(len(invalid_pin_0))
print(len(invalid_pin_with_citystate))
#print(len(invalid_pin_wout_city_state))
#print('==============================')
#print(len(invalid_pin_wout_citystate))
#print(len(invalid_pin_wout_cityORstate))
#print('===============================')
#print(len(invalid_pin_wout_city))
print(len(invalid_pin_wout_state))
print(len(invalid_pin_check_city))

60275
56197
2015
2063


In [6]:
invalid_pin_check_city.tail()

,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
2058,65055,r.k. ganapathi chettiar,"no 138, muthur roadkangayam",0,0,Tamil Nadu,91-4257-220492,0,0,www.rkgghee.com,Rkg Is A Leading Mfg Of Ghee And Is Known For ...
2059,80193,green world,"shree ram hospital, first floor, ksihan gate n...",0,0,Gujarat,0,91-9925777077,greenworldpots@gmail.com,www.greenworldpots.com,"Manufacturer Of Balcony Railig Pot, Self Water..."
2060,97644,jeeva velu matriculation higher secondary school,mathur tiruvannamalai,0,0,Tamil Nadu,04175-238779 2366119,0,admission@jeevavelu.com,0,School
2061,531766,kaynes technology india pvt ltd,"23/25, belagola food ind. area, metagalli po m...",0,0,Karnatak,0,8214002800,0,www.kaynestechnology.net,"Lighting, Led's , Lamp, Accessories, Luminaire..."
2062,533286,"sristech metal casting(i) pvt., ltd","village - bhagwanpur, dera bassi, sas nagar",0,0,Punjab,0,91-7527078501,srishtechindia@gmail.com,www.srishtechindia.com,"Manufacturer, Suppliers And Exporter of Horizo..."


In [6]:
invalid_pin_check_city['ADD'] = invalid_pin_check_city['ADD'].replace('nan', '', regex=True) 
invalid_pin_check_city['ADD'] = invalid_pin_check_city['ADD'].replace('nil', '', regex=True) 

invalid_pin_check_city_with_add = invalid_pin_check_city[invalid_pin_check_city['ADD'].str.strip().astype(bool)].reset_index(drop=True)
invalid_pin_check_city_wout_add = invalid_pin_check_city[~invalid_pin_check_city['ADD'].str.strip().astype(bool)].reset_index(drop=True)

print(len(invalid_pin_check_city_with_add))  # add exists, no city, state may or maynot exist
print(len(invalid_pin_check_city_wout_add))  # no add, no city, state may or maynot exist 

665
1398


In [7]:
dff_1 = invalid_pin_check_city_with_add.copy()
dff_2 = invalid_pin_check_city_with_add.copy()
invalid_pin_check_city_with_add[300:305]

,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
300,545800,scientific mes technik pvt ltd.,b 14 pologround industrial estate,0,0,0,91-731-2422330,91-9926666047,info@scientificindia.com,0,"Oscilloscope(CRO),Digital Oscilloscope(DSO),Fu..."
301,545807,sicame india connectors private limited,"#96, sirukundram village, chengalpet taluk, ka...",0,0,0,91-44-37270300,91-8870930576,rajesh.khanna@sicame.in,0,"HTLS accessories, Transmission and Distributio..."
302,545814,soorya electro components.pvt. ltd,"7/1b3, muppandal, aralvaimozhi",0,0,0,0,91-9943099124,shaji@sooryaelectrocomponents.com,0,"CTPT, INDOOR CT, OUTDOOR CT, LTCT"
303,545820,stucke elektronik india pvt ltd,"merkurrung 26, hamburg germany",0,0,0,0,0,warten@stuckegroup.de,0,"Generator Protection , Feeder Protection , Mot..."
304,545824,surendra composites pvt ltd,"14-15, new industrial area phase-ii, mandideep...",0,0,0,0,91-9755597773,sales@surendra.co.in,0,We produce : 1) Densified laminated wood boar...


In [8]:
# 24/5
idd = 304
print(dff_1['ADD'][idd])
add = dff_1['ADD'][idd].split(',')
add_check = add[-3:]

print(add_check)
add_check = [re.sub(r"\s+nagar", 'nagar', x) for x in add_check]
add_check = [re.sub(r'[^A-Za-z\s]', ' ', x) for x in add_check]   # remove digits and punctuations
add_check = [re.sub(r'\W*\b\w{1,2}\b', '', x) for x in add_check]  # removes single or double digits sub-strings
add_check = [re.sub(r'\n', '', x) for x in add_check]             # removes '\n'
#add_check = [x.strip() for x in add_check]  # not required
add_check = ' '.join(add_check)
add_check = add_check.split()

print(add_check)




14-15, new industrial area phase-ii, mandideep, dist. raisen (m.p.) 
[' new industrial area phase-ii', ' mandideep', ' dist. raisen (m.p.) ']
['new', 'industrial', 'area', 'phase', 'mandideep', 'dist', 'raisen']


In [58]:
# 24/5
idd = 664
print(dff_1['ADD'][idd])
add = dff_1['ADD'][idd].split(',')
add_check = add[-3:]

print(add_check)
add_check = [re.sub(r"\s+nagar", 'nagar', x) for x in add_check]
add_check = [re.sub(r'[^A-Za-z\s]', ' ', x) for x in add_check]   # remove digits and punctuations
add_check = [re.sub(r'\W*\b\w{1,2}\b', '', x) for x in add_check]  # removes single or double digits sub-strings
add_check = [re.sub(r'\n', '', x) for x in add_check]             # removes '\n'
#add_check = [x.strip() for x in add_check]  # not required
add_check = ' '.join(add_check)
add_check = add_check.split()

print(add_check)
#['bhagshala maidan', 'near rationing office dombivali  district thane']



village - bhagwanpur, dera bassi, sas nagar 
['village - bhagwanpur', ' dera bassi', ' sas nagar ']
['village', 'bhagwanpur', 'dera', 'bassi', 'sasnagar']


In [149]:
# new try on 24/5
a = []
b = []
c = []
d = []
e = []
f = []


print('==============================')

for i in (add_check):
    first_letter = i[0:3]
    cities = [c for c in city_list if c.startswith(first_letter)]
    m_ratio_1 = process.extract(i, cities, limit=2 ) #, scorer = fuzz.partial_ratio)
#    m_ratio_1 = process.extractOne(i, cities) #, scorer = fuzz.partial_ratio)
    a.append(m_ratio_1)
    m_ratio_2 = process.extract(i, cities,limit=2, scorer = fuzz.partial_ratio)
#    m_ratio_2 = process.extractOne(i, cities, scorer = fuzz.partial_ratio)
    b.append(m_ratio_2)
    m_ratio_3 = process.extract(i, cities,limit=2, scorer = fuzz.token_sort_ratio)
#    m_ratio_3 = process.extractOne(i, cities, scorer = fuzz.token_sort_ratio)
    c.append(m_ratio_3)
    m_ratio_4 = process.extract(i, cities,limit=2, scorer = fuzz.token_set_ratio)
#    m_ratio_4 = process.extractOne(i, cities, scorer = fuzz.token_set_ratio)
    d.append(m_ratio_4)
    m_ratio_5 = process.extract(i, cities,limit=2, scorer = fuzz.partial_token_sort_ratio)
#    m_ratio_5 = process.extractOne(i, cities, scorer = fuzz.partial_token_sort_ratio)
    e.append(m_ratio_5)
    m_ratio_6 = process.extract(i, cities,limit=2, scorer = fuzz.partial_token_set_ratio)
#    m_ratio_6 = process.extractOne(i, cities, scorer = fuzz.partial_token_set_ratio)
    f.append(m_ratio_6)
    
print('normal = ', a)
print('====================================')
print('partial_ratio = ', b)
print('====================================')
print('token_sort_ratio = ', c)
print('====================================')
print('token_set_ratio = ', d)
print('====================================')
print('partial_token_sort_ratio = ', e)
print('====================================')
print('partial_token_set_ratio = ', f)
    


normal =  [[('lucknow', 100), ('lucknow', 100)], [('indi', 89), ('indi', 89)]]
partial_ratio =  [[('lucknow', 100), ('lucknow', 100)], [('indi', 100), ('indi', 100)]]
token_sort_ratio =  [[('lucknow', 100), ('lucknow', 100)], [('indi', 89), ('indi', 89)]]
token_set_ratio =  [[('lucknow', 100), ('lucknow', 100)], [('indi', 89), ('indi', 89)]]
partial_token_sort_ratio =  [[('lucknow', 100), ('lucknow', 100)], [('indi', 100), ('indi', 100)]]
partial_token_set_ratio =  [[('lucknow', 100), ('lucknow', 100)], [('indi', 100), ('indi', 100)]]


In [57]:
# new try on 24/5
ratio_1_list_city = []
ratio_1_list_score = []
ratio_2_list = []

print('==============================')

for i in (add_check):
    first_letter = i[0:3]
    cities = [c for c in city_list if c.startswith(first_letter)]
    m_ratio_1 = process.extractOne(i, cities) #, scorer = fuzz.partial_ratio)
    
    m_ratio_2 = process.extractOne(i, cities, scorer = fuzz.token_sort_ratio)
    if m_ratio_1 != None and m_ratio_1[1] >90:
        
        ratio_1_list_city.append(m_ratio_1[0])
        ratio_1_list_score.append(m_ratio_1[1])
    ratio_2_list.append(m_ratio_2)
print(ratio_1_list_city)
print(ratio_1_list_score)
print(ratio_2_list)

if len(ratio_1_list_city) == 0 or len(ratio_1_list_city) > 2:
    dff_1['CITY'][idd] = 'Not Known'

if len(ratio_1_list_city) == 2:
    city_master_index_1 = city_master.index[city_master['place'] == ratio_1_list_city[0]]
    city_master_index_2 = city_master.index[city_master['place'] == ratio_1_list_city[1]]
    city_master_state_list = [city_master['state'][city_master_index_1]]
    # first check if file contains state :
    if dff_1['STATE'][idd] == 0:
        city_master_test_list = 
    

"""
ratio_1_clean = [x for x in ratio_1_list if x != None]

city_max = max(ratio_1_clean, key = (lambda val: val[1]), default=-1) # default=-1 is to avoid the raise exception ValueError when argument is empty list
city_max_index = [i for i in range(len(ratio_1_clean)) if ratio_1_clean[i][1] == city_max[1]]
found_city_list = [ratio_1_clean[x] for x in city_max_index]

print(found_city_list)
"""
print('=====================')

[('mandideep', 100), ('raisen', 100)]
[('newai', 75), ('indri', 67), ('areraj', 80), ('phagi', 60), ('mandideep', 100), None, ('raisen', 100)]


In [31]:
# new try on 24/5
ratio_1_list = []
ratio_2_list = []

print('==============================')

for i in (add_check):
    first_letter = i[0:3]
    cities = [c for c in city_list if c.startswith(first_letter)]
    m_ratio_1 = process.extractOne(i, cities) #, scorer = fuzz.partial_ratio)
    m_ratio_2 = process.extractOne(i, cities, scorer = fuzz.token_sort_ratio)
    if m_ratio_1 != None and m_ratio_1[1] >90:
        
        ratio_1_list.append(m_ratio_1)
    
    ratio_2_list.append(m_ratio_2)
print(ratio_1_list)
print(ratio_2_list)



[('mandideep', 100), ('raisen', 100)]
[('newai', 75), ('indri', 67), ('areraj', 80), ('phagi', 60), ('mandideep', 100), None, ('raisen', 100)]


In [35]:
print(type(ratio_1_list[2]))

<class 'NoneType'>


In [ ]:
# 24/5

for i in range(len(invalid_pin_check_city_with_add)):
    add = dff_1['ADD'][idd].split(',')                       # split address on ','
    add_check = add[-2:]                                     # take last 2 splits since city/states are at the end of an address
    add_check = [re.sub(r"\s+nagar", 'nagar', x) for x in add_check]  # merge 'nagar' with previus word if in same split
    add_check = [re.sub(r'[^A-Za-z\s]', ' ', x) for x in add_check]   # remove digits and punctuations
    add_check = [re.sub(r'\W*\b\w{1,2}\b', '', x) for x in add_check]  # removes single or double digits sub-strings
    add_check = [re.sub(r'\n', '', x) for x in add_check]             # removes '\n'
    add_check = ' '.join(add_check)                                   # make a single list
    add_check = add_check.split()
    
    for i in (add_check):
        first_letter = i[0:3]
        cities = [c for c in city_list if c.startswith(first_letter)]
        m_ratio_1 = process.extractOne(i, cities) #, scorer = fuzz.partial_ratio)
        m_ratio_2 = process.extractOne(i, cities, scorer = fuzz.token_sort_ratio)


In [13]:


# in the regex of 1st line : ^ means except, a-z means only letters without (puntuations and digits), \s only single space
# in the regex of 2nd line : keep only strings having length > 2

invalid_pin_check_city_with_add['ADD'] = invalid_pin_check_city_with_add['ADD'].apply(lambda x: re.sub('[^a-z\s]', ' ', x))

invalid_pin_check_city_with_add['ADD'] = invalid_pin_check_city_with_add['ADD'].apply(lambda x: re.sub(r'\W*\b\w{1,2}\b', '', x))
invalid_pin_check_city_with_add['ADD'] = invalid_pin_check_city_with_add['ADD'].replace('\n','', regex=True)  # remove '\n'


invalid_pin_check_city_with_add.tail()                                                                

,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
660,65055,r.k. ganapathi chettiar,muthur roadkangayam,0,0,Tamil Nadu,91-4257-220492,0,0,www.rkgghee.com,Rkg Is A Leading Mfg Of Ghee And Is Known For ...
661,80193,green world,shree ram hospital first floor ksihan gate ...,0,0,Gujarat,0,91-9925777077,greenworldpots@gmail.com,www.greenworldpots.com,"Manufacturer Of Balcony Railig Pot, Self Water..."
662,97644,jeeva velu matriculation higher secondary school,mathur tiruvannamalai,0,0,Tamil Nadu,04175-238779 2366119,0,admission@jeevavelu.com,0,School
663,531766,kaynes technology india pvt ltd,belagola food ind area metagalli mysouru,0,0,Karnatak,0,8214002800,0,www.kaynestechnology.net,"Lighting, Led's , Lamp, Accessories, Luminaire..."
664,533286,"sristech metal casting(i) pvt., ltd",village bhagwanpur dera bassi sas nagar,0,0,Punjab,0,91-7527078501,srishtechindia@gmail.com,www.srishtechindia.com,"Manufacturer, Suppliers And Exporter of Horizo..."


In [10]:
city_master = pd.read_excel('place_list.xlsx', header=0)
city_list = city_master['place'].tolist()
city_master.head(3)

,place,district,state,std_code,region
0,a.thirumuruganpoondi,coimbatore,tamil nadu,NaN,NaN
1,a.vellalapatti,madurai,tamil nadu,NaN,NaN
2,aadityana,porbandar,gujarat,NaN,NaN


In [19]:
a = []
b = []
c = []
d = []
e = []
f = []

add = invalid_pin_check_city_with_add['ADD'][664]
add_list = add.split()
print('address = ', dff['ADD'][664])
print(add_list)
print('==============================')

for i in (add_list):
    first_letter = i[0:3]
    cities = [c for c in city_list if c.startswith(first_letter)]
    m_ratio_1 = process.extract(i, cities, limit=2 ) #, scorer = fuzz.partial_ratio)
    a.append(m_ratio_1)
    m_ratio_2 = process.extract(i, cities,limit=2, scorer = fuzz.partial_ratio)
    b.append(m_ratio_2)
    m_ratio_3 = process.extract(i, cities,limit=2, scorer = fuzz.token_sort_ratio)
    c.append(m_ratio_3)
    m_ratio_4 = process.extract(i, cities,limit=2, scorer = fuzz.token_set_ratio)
    d.append(m_ratio_4)
    m_ratio_5 = process.extract(i, cities,limit=2, scorer = fuzz.partial_token_sort_ratio)
    e.append(m_ratio_5)
    m_ratio_6 = process.extract(i, cities,limit=2, scorer = fuzz.partial_token_set_ratio)
    f.append(m_ratio_6)
    
print('normal = ', a)
print('====================================')
print('partial_ratio = ', b)
print('====================================')
print('token_sort_ratio = ', c)
print('====================================')
print('token_set_ratio = ', d)
print('====================================')
print('partial_token_sort_ratio = ', e)
print('====================================')
print('partial_token_set_ratio = ', f)
    

address =  village - bhagwanpur, dera bassi, sas nagar 
['village', 'bhagwanpur', 'dera', 'bassi', 'sas', 'nagar']
normal =  [[('villiappally', 64), ('vilangudi', 62)], [('bhagalpur', 84), ('bhawanathpur', 82)], [('dera baba nanak', 90), ('dera bassi', 90)], [('bassi', 100), ('bassi pathana', 90)], [('sasan gir', 90), ('sasaram', 90)], [('nagar', 100), ('nagari', 91)]]
partial_ratio =  [[('vilangudi', 71), ('villiappally', 71)], [('bhagalpur', 78), ('bhargain', 75)], [('dera baba nanak', 100), ('dera bassi', 100)], [('bassi', 100), ('bassi pathana', 100)], [('sasan gir', 100), ('sasaram', 100)], [('nagar', 100), ('nagarakata', 100)]]
token_sort_ratio =  [[('vilangudi', 62), ('villupuram', 59)], [('bhagalpur', 84), ('bhawanathpur', 82)], [('derapur', 73), ('dergaon', 73)], [('bassi', 100), ('bassein', 83)], [('sasni', 75), ('sasti', 75)], [('nagar', 100), ('nagari', 91)]]
token_set_ratio =  [[('vilangudi', 62), ('villupuram', 59)], [('bhagalpur', 84), ('bhawanathpur', 82)], [('dera baba

In [33]:
a = []
b = []
c = []
d = []
e = []
f = []

check_index = 657
add = invalid_pin_check_city_with_add['ADD'][check_index]
add_list = add.split()
print('address = ', dff['ADD'][check_index])
print(add_list)
print('==============================')

for i in (add_list):
    first_letter = i[0:3]
    cities = [c for c in city_list if c.startswith(first_letter)]
    m_ratio_1 = process.extractOne(i, cities) #, scorer = fuzz.partial_ratio)
    a.append(m_ratio_1)
    m_ratio_2 = process.extractOne(i, cities, scorer = fuzz.partial_ratio)
    b.append(m_ratio_2)
    m_ratio_3 = process.extractOne(i, cities, scorer = fuzz.token_sort_ratio)
    c.append(m_ratio_3)
    m_ratio_4 = process.extractOne(i, cities, scorer = fuzz.token_set_ratio)
    d.append(m_ratio_4)
    m_ratio_5 = process.extractOne(i, cities, scorer = fuzz.partial_token_sort_ratio)
    e.append(m_ratio_5)
    m_ratio_6 = process.extractOne(i, cities, scorer = fuzz.partial_token_set_ratio)
    f.append(m_ratio_6)
    
print('normal = ', a)
print('====================================')
print('partial_ratio = ', b)
print('====================================')
print('token_sort_ratio = ', c)
print('====================================')
print('token_set_ratio = ', d)
print('====================================')
print('partial_token_sort_ratio = ', e)
print('====================================')
print('partial_token_set_ratio = ', f)
    

address =  village jagat khanna
['village', 'jagat', 'khanna']
normal =  [('villiappally', 64), ('jagatballavpur', 90), ('khanna', 100)]
partial_ratio =  [('vilangudi', 71), ('jagatballavpur', 100), ('khanna', 100)]
token_sort_ratio =  [('vilangudi', 62), ('jagtaj', 73), ('khanna', 100)]
token_set_ratio =  [('vilangudi', 62), ('jagtaj', 73), ('khanna', 100)]
partial_token_sort_ratio =  [('vilangudi', 71), ('jagatballavpur', 100), ('khanna', 100)]
partial_token_set_ratio =  [('vilangudi', 71), ('jagatballavpur', 100), ('khanna', 100)]


In [34]:
idd = city_master.index[city_master['place'] == 'raisen']  # 4 spaces
idd

Int64Index([5794, 5795], dtype='int64')

In [33]:
city_master[4188:4189]

,place,district,state,std_code,region
4188,mandideep,raisen,madhya pradesh,NaN,NaN


In [35]:
city_master[5794:5796]

,place,district,state,std_code,region
5794,raisen,raisen,madhya pradesh,NaN,NaN
5795,raisen,NaN,madhya pradesh,7482.0,NaN


In [84]:
#df[df['A']==5].index.item()
#df.index.get_loc(df.index[df['b'] == 5][0])
#np.where(df['c']==6)[0]
i1 = city_master[city_master['place'] == 'mandideep'].index.item()
print(i1)
#i2 = city_master.index[city_master['place'] == 'raisen'].item()
#i2 = city_master[city_master['place'] == 'raisen'].index.item()
i2 = np.where(city_master['place']=='raisen')
#i3 = i1+i2
print(type(i2))
#print(i3)
#c_m_s_list = [city_master['district']i1, city_master['district']i2, city_master['state']i1, city_master['state']i2]
#print(c_m_s_list)
city_master['district'][i1]


4188
<class 'tuple'>


'raisen'

In [36]:
dff[657:658]

,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
657,81496,vishwakarma industries,village jagat khanna,0,0,Himachal pradesh,91-1795-265318,0,vishwakarna.hp1@gmail.com,www.vishwakarmagroups.com,Mfg Of Clean Room Equipment And Ss Fabricated ...


In [29]:
ratio = process.extract('dera bassi', add_list, scorer = fuzz.token_set_ratio)
print(ratio)

[('dera', 100), ('bassi', 100), ('sas', 31), ('bhagwanpur', 30), ('nagar', 27)]


In [12]:
import difflib

In [38]:
correct_roadnames = ["Aljunied Avenue 2", "Aljunied Avenue 1", "Aljunied Avenue 3" ]
correct_r = ["Aljunied", "Aljunid", "Aljuniid" ]
print(process.extract("Aljuneid Avenue 1", correct_roadnames))
print(process.extract("Aljuneid", correct_r))
print(difflib.get_close_matches("Aljuneid", correct_r))

[('Aljunied Avenue 1', 94), ('Aljunied Avenue 2', 88), ('Aljunied Avenue 3', 88)]
[('Aljunid', 93), ('Aljunied', 88), ('Aljuniid', 88)]
['Aljunid', 'Aljuniid', 'Aljunied']


In [37]:
d = fuzz.ratio('hariana', 'haryana')
e = fuzz.partial_ratio('hariana', 'haryana')
f = fuzz.token_sort_ratio('hariana', 'haryana')
g = fuzz.token_set_ratio('hariana', 'haryana')

choices = ['horiana', 'haryana', 'haryanna', 'heryana', 'heriana'] #, 'gas', 'godown', 'bhayandar', 'kashi', 'mira', 'road', 'bhayandar', 'east']
h = process.extract('hariana', choices)
i = difflib.get_close_matches('hariana', choices, n=3, cutoff = .85)

print(d, e, f, g)
print(h)
print(i)

86 86 86 86
[('horiana', 86), ('haryana', 86), ('heriana', 86), ('haryanna', 80), ('heryana', 71)]
['horiana', 'heriana', 'haryana']


In [30]:
d = fuzz.ratio('miraroad', 'mira road')
e = fuzz.partial_ratio('miraroad', 'mira road')
f = fuzz.token_sort_ratio('miraroad', 'mira road')
g = fuzz.token_set_ratio('miraroad', 'mira road')

choices = ['dera bassi', 'bassi', 'industrial', 'estate', 'behind', 'gas', 'godown', 'bhayandar', 'kashi', 'mira road', 'bhayandar', 'east']

h = process.extract('miraroad', choices, limit=3)
i = difflib.get_close_matches('miraroad', choices, cutoff=.9)
print(d, e, f, g)
print(h)
print(i)

94 88 94 94
[('mira road', 94), ('bhayandar', 35), ('bhayandar', 35)]
['mira road']


In [10]:
nomi = pgeocode.Nominatim('in')
oout = nomi.query_postal_code("440022")
print(oout[2])
print(oout[5])
print(oout[7])

Laxmi Nagar (Nagpur), Trimurti Nagar, Ranapratap Nagar, Shradhananpeth
Nagpur
Nagpur


In [12]:
city_master = pd.read_excel('place_list.xlsx', header=0)
city_master.head(3)

,place,district,state,std_code,region
0,a.thirumuruganpoondi,coimbatore,tamil nadu,NaN,NaN
1,a.vellalapatti,madurai,tamil nadu,NaN,NaN
2,aadityana,porbandar,gujarat,NaN,NaN


In [291]:
add = invalid_pin_wout_city_state['ADD'][112].split()
print(len(add))
print(add)


4
['nariyal', 'bazar', 'meston', 'road']


In [214]:
city_list = city_master['place'].tolist()
for i in (add):
    first_letter = i[0:2]
    cities = [c for c in city_list if c.startswith(first_letter)]
    m_ratio = process.extractOne(i, cities) #, scorer = fuzz.partial_ratio)
    print(m_ratio)

('bareilly', 100)
None
('indergarh', 72)
('colvale', 71)


In [230]:
idd = invalid_pin_wout_city_state.index[invalid_pin_wout_city_state['ADD'] == '']  # no space
idd 

Int64Index([], dtype='int64')

In [232]:
idd = invalid_pin_wout_city_state.index[invalid_pin_wout_city_state['ADD'] == ' ']  # 1 space
idd

Int64Index([   9,   10,   11,  122,  127,  128,  143,  147,  148,  149,
            ...
            2272, 2273, 2280, 2285, 2311, 2342, 2353, 2354, 2355, 3585],
           dtype='int64', length=362)

In [233]:
idd = invalid_pin_wout_city_state.index[invalid_pin_wout_city_state['ADD'] == '  ']  # 2 spaces
idd

Int64Index([3584], dtype='int64')

In [234]:
idd = invalid_pin_wout_city_state.index[invalid_pin_wout_city_state['ADD'] == '   ']  # 3 spaces
idd

Int64Index([  12,   14, 3163, 3167, 3169, 3171, 3173, 3176, 3181, 3182, 3189,
            3197, 3200, 3204, 3206, 3211, 3213, 3303, 3304, 3305, 3306, 3307,
            3308, 3309, 3310, 3311, 3312, 3313, 3314, 3315, 3316, 3317, 3318,
            3319, 3320, 3321, 3322, 3323, 3324, 3352, 3353, 3354, 3355, 3357,
            3358, 3359, 3361, 3362, 3363, 3590],
           dtype='int64')

In [236]:
idd = invalid_pin_wout_city_state.index[invalid_pin_wout_city_state['ADD'] == '    ']  # 4 spaces
idd

Int64Index([3365, 3366, 3367, 3368, 3370, 3372, 3373, 3374, 3375, 3376, 3384], dtype='int64')

In [238]:
idd = invalid_pin_wout_city_state.index[invalid_pin_wout_city_state['ADD'] == '     ']  # 5 spaces
idd

Int64Index([], dtype='int64')

In [239]:
idd = invalid_pin_wout_city_state.index[invalid_pin_wout_city_state['ADD'] == '      ']  # 6 spaces
idd 

Int64Index([], dtype='int64')

In [349]:
invalid_pin_wout_city_state['ADD'] = invalid_pin_wout_city_state['ADD'].replace('nan', '', regex=True) 
invalid_pin_wout_city_state['ADD'] = invalid_pin_wout_city_state['ADD'].replace('nil', '', regex=True) 

df = invalid_pin_wout_city_state[invalid_pin_wout_city_state['ADD'].str.strip().astype(bool)]
df_1 = invalid_pin_wout_city_state[~invalid_pin_wout_city_state['ADD'].str.strip().astype(bool)]

In [355]:
print(len(invalid_pin_wout_city_state))
print(len(df))
print(len(df_1))

4078
2335
1743


In [353]:
df_1['ADD'][4000]

''

In [282]:
7 = 'nan', 9 = [], 4000 = 'nil', 

SyntaxError: cannot assign to literal (<ipython-input-282-1e4d1f265ceb>, line 1)

In [297]:
a = np.nan

In [329]:
len(invalid_pin_wout_city_state)

4078